In [ ]:
#r "nuget: FsChat.Interactive, 0.1.0-beta2"

#r "nuget: dotenv.net, 3.2.0"
open dotenv.net
DotEnv.Load(DotEnvOptions(envFilePaths=[ "./.env" ]))
open FsChat

Restore sources /home/peter/work/FsChat/nuget.local Installed Packages dotenv.net, 3.2.0 FsChat.Interactive, 0.1.0-beta2

Loading extension script from `/home/peter/.nuget/packages/fschat.interactive/0.1.0-beta2/interactive-extensions/dotnet/extension.dib`

FsChat is ready.

In [ ]:
let style = """<style>
    .chat-bubble {
        display: flex;
        flex-direction: column;
        width: 100%;
        max-width: 400px;
        margin: 0 auto;
    }
    .chat-bubble>p {
        max-width: 70%;
        padding: 10px 15px;
        margin: 10px;
        border-radius: 15px;
        position: relative;
        word-wrap: break-word;
    }
    .chat-bubble.left>p {
        align-self: flex-start;
        background-color: #e1bee7;
        border: 2px solid #ce93d8;
        color: #6a1b9a;
    }
    .chat-bubble.right>p {
        align-self: flex-end;
        background-color: #c8e6c9;
        border: 2px solid #a5d6a7;
        color: #2e7d32;
    }
</style>"""

In [ ]:
let agent1 = Chat(
    model=OpenAI.gpt4o_mini,
    renderer=NotebookRenderer(props="class='chat-bubble left'", css=style),
    prompt=[
        System """
            You're playing the 20 questions game.
            Your role is to ask 'Is it a ...' questions with Yes-or-No answers
            in order to narrow down your options and guess the word that I'm thinking of.
            ----
            Hint: You're guessing an animal.
        """
    ])

let agent2 = Chat(
    model=OpenAI.gpt4o_mini,
    renderer=NotebookRenderer(props="class='chat-bubble right'", css=style),
    prompt=[
        System """
            You're playing the 20 questions game.
            Your role is the one who thinks of a word and responds to my questions
            with simple "yes" or "no" answers (no additional text).
            If necessary, you can also say "Both yes and no" or "I can’t answer that".
            Once I guess the correct word or correct class of animal species,
            respond with “CORRECT.”
            ----
            The word I'll be trying to guess is: parrot.
        """
    ])

let rec play timesLeft (text:string) =
    if timesLeft=0 then
        printfn "Game over"
    else
        let quess = agent1.send(text).text
        let assess = agent2.send(quess).text
        if assess.Contains "CORRECT." then
            ()
        else
            play (timesLeft-1) assess

play 20 "I have a word! Which word is it? Ask the first question."

Is it a mammal?

No.

Is it a bird?

Yes.

Is it a flightless bird?

No.

Is it a bird that is commonly kept as a pet?

Yes.

Is it a type of parrot?

CORRECT.

In [ ]:
agent2.messages

index value 0 { role = system\n content =\n "You're playing the 20 questions game.\nYour role is the one who thinks of a word and responds to my questions\nwith simple "yes" or "no" answers (no additional text).\nIf necessary, you can also say "Both yes and no" or "I can’t answer that".\nOnce I guess the corre... role system content You're playing the 20 questions game.
Your role is the one who thinks of a word and responds to my questions
with simple "yes" or "no" answers (no additional text).
If necessary, you can also say "Both yes and no" or "I can’t answer that".
Once I guess the correct word or correct class of animal species,
respond with “CORRECT.”
----
The word I'll be trying to guess is: parrot. 1 { role = user\n content = "Is it a mammal?" } role user content Is it a mammal? 2 { role = assistant\n content = "No." } role assistant content No. 3 { role = user\n content = "Is it a bird?" } role user content Is it a bird? 4 { role = assistant\n content = "Yes." } role assistant content Yes. 5 { role = user\n content = "Is it a flightless bird?" } role user content Is it a flightless bird? 6 { role = assistant\n content = "No." } role assistant content No. 7 { role = user\n content = "Is it a bird that is commonly kept as a pet?" } role user content Is it a bird that is commonly kept as a pet? 8 { role = assistant\n content = "Yes." } role assistant content Yes. 9 { role = user\n content = "Is it a type of parrot?" } role user content Is it a type of parrot? 10 { role = assistant\n content = "CORRECT." } role assistant content CORRECT.